In this experiment, we will cover:

- Uncertainty in deep learning
- Dropout and Batch normalization
- Ensembling

- Based on the course -
  * Deep Learning and Applied AI

  * Lecturer: Prof. Emanuele Rodolà

  * Sapienza University of Rome

##Import dependencies (run the following cells)

In [ ]:
# @title import dependencies

from typing import Mapping, Union, Optional

import numpy as np
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import plotly.graph_objects as go
import plotly.express as px
import torchvision
from torchvision import datasets, models, transforms

import os
import pickle
from tqdm.notebook import tqdm

from __future__ import print_function, division

In [ ]:
# @title reproducibility stuff

import random
torch.manual_seed(42)
np.random.seed(42)
random.seed(0)

torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True  # Note that this Deterministic mode can have a performance impact
torch.backends.cudnn.benchmark = False

##Uncertainty in deep learning and two popular regularization techniques

In this last section we will breafly discute about *uncertainty* in deep learning, an inescapable concept whenever we aim to extrapolate a general rule from finite data.

We will also experiment with two popular *regularization* methods, dropout and batch normalization.

Surprisingly enough these two arguments fits well in the same section, since a very effective and simple way to model uncertainty in deep learning is through dropout.




#### Uncertainty in deep learning

> "*In almost all circumstances, and at all times,
we find ourselves in a state of uncertainty.*
>
>*Uncertainty in every sense.*
>
>*Uncertainty about actual situations, past and present (this might stem from either a lack of knowledge and information, or from the incompleteness and unreliability of the information at our disposal, either ours or someone else's, to provide a convincing recollection of these situations.) [...]*
>
>*Uncertainty in the face of decisions: more than ever in this case, compounded by the fact that decisions have to be based on knowledge of the actual situation, which is itself uncertain, to be guided by the prevision of uncontrollable events, and to aim for certain desirable effects of the decisions themselves, these also being uncertain.*"

>Bruno de Finetti *Theory of Probability: A critical introductory treatment*, Chapter 2



Despite representing model uncertainty in deep learning is of crucial importance -- think about medical applications or self-driving cars -- standard DNNs do not provide such information.

The $\pm$ symbol denoting the confidence interval of predictions is rare in deep learning papers, even if a prediction of a DNN on a test sample is everything except certain; uncertainty does not origins only in intrinsic stochastic processes (such as a radioactive decay or the roll of a dice), but *also when we have a lack of knowledge*, when we try to make a bet on something that is out of our ground truth, such as a test sample.

Notice that the probability interpretation of a softmax output does not solve the problem, **a model can be uncertain in its prediction even with a softmax output close to 1** as we will see.

Today we will explore a very simple idea to model uncertainty in deep learning through dropout, following the works of Gal and Ghahramani:
- [*Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning*](https://arxiv.org/abs/1506.02142)
-[*Bayesian Convolutional Neural Networks with Bernoulli Approximate Variational Inference*](https://arxiv.org/abs/1506.02158)

#### Dropout and Batch Normalization; two common regularization methods

As seen in lecture, regularizers are general methods to reduce overfitting and thus improve generalization.

Regularization methods are based on general considerations about the learning algorithm, their ultimate objective is to reduce the number of free parameters.

Today we will experiment with
- **Dropout**: Training an ensemble of neural networks parametrizing each model by dropping random units from a single large network.
- **Batch Normalization**: Normalizing the activations of hidden layers as we do with the input data, allowing an easy learning of the identity function for the hidden layer.

### Training a bunch of models on CIFAR10

Today we will train several models on CIFAR10, experimenting with the effects of regularization and trying to say something about the uncertainty of our predictions.

Let's download and normalize the dataset...

In [ ]:
train_transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_transform)


100%|██████████| 170498071/170498071 [00:13<00:00, 12903717.64it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


...and then prepare the dataloaders.

In [ ]:
batch_size = 32
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
print('train set size: {}'.format(len(trainset)))
print('test set size: {}'.format(len(testset)))


trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=4)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=4)


train set size: 50000
test set size: 10000


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


As always we want to visualize some samples. Make your own prediction on each one.

In [ ]:
# @title Visualize samples function

def visualize_samples(inputs, title=None):
    """
    Visualization of transformed samples, a standard call:
        inputs, classes = next(iter(dataloaders['train']))
        visualize_samples(inputs)
    Arguments:
    batch_of_samples -- a batch from the dataloader; a PyTorch tensor of shape (batch_size, 3, 224, 224)

    Return:
    None (A nice plot)
    """

    # Make a grid from batch
    inp = torchvision.utils.make_grid(inputs, nrow=12)

    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)  # plotly accepts the colour information both in the 0-1 range and in the 0-255 range
    fig = px.imshow(inp, title=title)
    fig.show()


# Get a batch of training data
inputs = [trainset[i][0] for i in range(4)]
class_idx = [trainset[i][1] for i in range(4)]


visualize_samples(inputs, title=f'Make your prediction, which is the label of each image? The labels are<br> {[x for x in classes]}')

# Solution
# print(f'Ground truth: {[classes[x] for x in class_idx]}')

> **EXERCISE** Be conscious about your uncertainty on these predictions, how much would you bet on your guesses on each image?

For the purpose of our experiments we will work with a very simple CNN architecture, similar to the famous LeNet of 1998 by Yann Lecun, a time when there was not dropout nor batch normalization.

We will try 4 + 1 different models:
- Lenet  without dropout nor batch normalization (`VanillaLeNet`).
- Lenet with standard dropout after each fully connected layer (`StdDropoutLeNet`).
- Lenet with dropout2d (zeroing entire channels) after each convolutional layer and standard dropout after each fully connected layer (`FullDropoutLeNet`).
- LeNet with Batch Normalization after each layer (`BatchNormLeNet`)

To be less boring than usual today we will use **SELUs** (scaled exponential linear units); a very exotic non-linearity from the crowded [zoo of activation functions](https://pytorch.org/docs/stable/nn.functional.html#non-linear-activation-functions). If Relu is the lion, Selu could be a platypus; spot its definition in the PyTorch documentation.


Selu has been proposed in [*Self Normalizing Neural Networks*](https://arxiv.org/abs/1706.02515) and it is supported by a mammoth ablation study on more than 100 machine learning tasks and a 90-pages-long appendix full of calculations. The main point of SELUs is to induce self-normalizing properties without the necessity of batch normalization. This is of special interest for FNNs (fully connected networks) with many layers, which suffer most the perturbations induced by batch normalization.





In [ ]:
# We will perform the final softmax out of the net classes to be ready for a future experiment

# Lenet without dropout
class VanillaLeNet(nn.Module):
    def __init__(self):
        super(VanillaLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 192, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(192, 192, 5, padding=2)
        self.fc1 = nn.Linear(192 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.selu(self.pool(self.conv1(x)))
        x = F.selu(self.pool(self.conv2(x)))
        x = x.view(-1, 192 * 8 * 8)
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(x))
        x = self.fc3(x)
        return x


# Lenet with dropout after fully connected layers
class StdDropoutLeNet(nn.Module):
    def __init__(self):
        super(StdDropoutLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 192, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(192, 192, 5, padding=2)
        self.fc1 = nn.Linear(192 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(p=0.3)

    def forward(self, x):
        x = F.selu(self.pool(self.conv1(x)))
        x = F.selu(self.pool(self.conv2(x)))
        x = x.view(-1, 192 * 8 * 8)
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(self.dropout(x)))
        x = self.fc3(self.dropout(x))
        return x


# Lenet with dropout also after convolutional layers
class FullDropoutLeNet(nn.Module):
    def __init__(self):
        super(FullDropoutLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 192, 5, padding=2)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(192, 192, 5, padding=2)
        self.fc1 = nn.Linear(192 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 10)
        self.dropout = nn.Dropout(p=0.3)
        self.dropout2d = nn.Dropout2d(p=0.3)

    def forward(self, x):
        x = F.selu(self.pool(self.dropout2d(self.conv1(x)))) # dropout 2D!
        x = F.selu(self.pool(self.dropout2d(self.conv2(x))))
        x = x.view(-1, 192 * 8 * 8)
        x = F.selu(self.fc1(x))
        x = F.selu(self.fc2(self.dropout(x)))
        x = self.fc3(self.dropout(x))
        return x


# Lenet with batch normalization
class BatchNormLeNet(nn.Module):
    def __init__(self):
        super(BatchNormLeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 192, 5, padding=2)
        self.bn1 = nn.BatchNorm2d(192)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(192, 192, 5, padding=2)
        self.bn2 = nn.BatchNorm2d(192)
        self.fc1 = nn.Linear(192 * 8 * 8, 1024)
        self.bnf1 = nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 256)
        self.bnf2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        # Since Selu purpose is to make batchnorm unnecesary, in this architecture we are going to use standard Relu
        x = F.relu(self.pool(self.bn1(self.conv1(x))))
        x = F.relu(self.pool(self.bn2(self.conv2(x))))
        x = x.view(-1, 192 * 8 * 8)
        x = F.relu(self.bnf1(self.fc1(x)))
        x = F.relu(self.bnf2(self.fc2(x)))
        x = self.fc3(x)
        return x

And the + 1?

Here we are:

- Lenet with *Monte Carlo dropout*, with the very same architecture of `FullDropoutLeNet` during training, but different at test time; instead of multiplying the output of each neuron by its dropout coefficient, we continue to mask out neurons during evaluation, but collecting several predictions of the same samples and then taking the average, i.e. the same test sample is given to *different models of the dropout ensemble*. This will also allow us to reason about the uncertainty of the prediction. (`MonteCarloDropoutLeNet`)

Let's define two functions to wrap up the training and test pipelines. The test one should take into account the different test modality of `MonteCarloDropoutLeNet`.

In [ ]:
# We want to print the training loss every log_freq batches
log_freq = len(trainset)//batch_size  # default

def train(epoch, net, optimizer, loss_func, log_freq=log_freq):
    running_loss = 0.0
    for i, data in enumerate(trainloader, start=1):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if (i) % log_freq == 0:    # print every log_freq mini-batches
            print('[Epoch : %d, Iter: %5d] loss: %.3f' %
                  (epoch + 1, i, running_loss / log_freq))
            running_loss = 0.0
    return running_loss / log_freq

def test(net, is_MCDO=False, train_data=False):
    if train_data:
        print("Accuracy on training data")
        dataloader = trainloader
    else:
        print("Accuracy on test data")
        dataloader = testloader
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for batch_idx, data in enumerate(dataloader):
            if batch_idx == len(testloader):
                break
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            output = 0
            if not is_MCDO:
                output = net(inputs)
            else:
                for i in range(20):  # number of different predictions
                    output += F.softmax(net(inputs), dim=1) / 20.
                output = torch.log(output)
            _, predicted = torch.max(output, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    for i in range(10):
        print('Accuracy of %5s : %.2f %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))

    test_score = np.mean([100 * class_correct[i] / class_total[i] for i in range(10)])
    print(test_score)
    return test_score

>⚠️ **WARNING** The following cell will start the training of the models, it will take about 1 hour and 30 minutes. You can go over it without training, the next section will load a pretrained model.

Each model is saved in the `SAVE_PATH` at the end of its training, if you want to store the trained models beyond any *Colab Runtime disconnected* you can mount your Google Drive and set `SAVE_PATH` inside your drive. To mount your Drive open the Files menu on the left (Folder icon).



In [ ]:
from tqdm.notebook import tqdm

run_training = True  #@param {type:"boolean"}
SAVE_PATH = '/content/'  #@param {type:"string"}

# the architecture of MonteCarloDropoutLeNet is the same of FullDropoutLeNet
lenets = [FullDropoutLeNet, StdDropoutLeNet, VanillaLeNet, BatchNormLeNet]

epoch_num = 75
test_freq = 5
losses = []
net_scores = {lenet.__name__ : [] for lenet in lenets}
net_scores['MonteCarloDropoutLeNet'] = []
net_tr_scores = {lenet.__name__ : [] for lenet in lenets}
net_tr_scores['MonteCarloDropoutLeNet'] = []
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if run_training:
    for lenet in lenets:
        print(lenet.__name__, 'training')
        net = lenet()
        net.to(device)

        learning_rate = 5e-4
        loss_func = nn.CrossEntropyLoss()
        optimizer = optim.Adam(net.parameters(), lr=learning_rate, weight_decay=0.0005, amsgrad=True)
        scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.75)

        for i in tqdm(range(epoch_num)):
            net.train()
            loss_avg = train(epoch=i, net=net, optimizer=optimizer, loss_func=loss_func)
            losses.append(loss_avg)
            scheduler.step()

            if (i+1) % test_freq == 0:
                if lenet.__name__ == 'FullDropoutLeNet':
                    print('FullDropoutLeNet TEST')
                    net.eval()
                    net_score = test(net)
                    net_scores['FullDropoutLeNet'].append(net_score)
                    net_tr_score = test(net, train_data=True)
                    net_tr_scores['FullDropoutLeNet'].append(net_tr_score)
                    print('MonteCarloDropoutLeNet TEST')
                    net.train()
                    net_score = test(net,is_MCDO=True)
                    net_scores['MonteCarloDropoutLeNet'].append(net_score)
                    net_tr_score = test(net, is_MCDO=True, train_data=True)
                    net_tr_scores['MonteCarloDropoutLeNet'].append(net_tr_score)
                else:
                    net.eval()
                    net_score = test(net)
                    net_scores[lenet.__name__].append(net_score)
                    net_tr_score = test(net, train_data=True)
                    net_tr_scores[lenet.__name__].append(net_tr_score)
        torch.save(net.state_dict(), SAVE_PATH + lenet.__name__ + '.pt')



FullDropoutLeNet training


[Epoch : 1, Iter:  1562] loss: 1.509
[Epoch : 2, Iter:  1562] loss: 1.129
[Epoch : 3, Iter:  1562] loss: 1.001
[Epoch : 4, Iter:  1562] loss: 0.928
[Epoch : 5, Iter:  1562] loss: 0.861
FullDropoutLeNet TEST
Accuracy on test data
Accuracy of plane : 77.40 %
Accuracy of   car : 75.00 %
Accuracy of  bird : 53.70 %
Accuracy of   cat : 49.80 %
Accuracy of  deer : 70.30 %
Accuracy of   dog : 54.60 %
Accuracy of  frog : 86.00 %
Accuracy of horse : 75.20 %
Accuracy of  ship : 79.10 %
Accuracy of truck : 79.90 %
70.1
Accuracy on training data
Accuracy of plane : 82.22 %
Accuracy of   car : 84.11 %
Accuracy of  bird : 64.15 %
Accuracy of   cat : 61.85 %
Accuracy of  deer : 78.38 %
Accuracy of   dog : 61.62 %
Accuracy of  frog : 88.80 %
Accuracy of horse : 84.77 %
Accuracy of  ship : 84.65 %
Accuracy of truck : 88.42 %
77.89582420664777
MonteCarloDropoutLeNet TEST
Accuracy on test data
Accuracy of plane : 78.10 %
Accuracy of   car : 78.70 %
Accuracy of  bird : 52.30 %
Accuracy of   cat : 53.90 %


[Epoch : 1, Iter:  1562] loss: 1.405
[Epoch : 2, Iter:  1562] loss: 1.009
[Epoch : 3, Iter:  1562] loss: 0.866
[Epoch : 4, Iter:  1562] loss: 0.776
[Epoch : 5, Iter:  1562] loss: 0.699
Accuracy on test data
Accuracy of plane : 83.50 %
Accuracy of   car : 84.80 %
Accuracy of  bird : 57.60 %
Accuracy of   cat : 55.90 %
Accuracy of  deer : 81.10 %
Accuracy of   dog : 54.60 %
Accuracy of  frog : 75.90 %
Accuracy of horse : 72.20 %
Accuracy of  ship : 80.80 %
Accuracy of truck : 83.50 %
72.99
Accuracy on training data
Accuracy of plane : 90.31 %
Accuracy of   car : 94.19 %
Accuracy of  bird : 73.74 %
Accuracy of   cat : 71.48 %
Accuracy of  deer : 91.57 %
Accuracy of   dog : 70.26 %
Accuracy of  frog : 83.32 %
Accuracy of horse : 83.60 %
Accuracy of  ship : 88.61 %
Accuracy of truck : 91.78 %
83.88636866347333
[Epoch : 6, Iter:  1562] loss: 0.630
[Epoch : 7, Iter:  1562] loss: 0.572
[Epoch : 8, Iter:  1562] loss: 0.524
[Epoch : 9, Iter:  1562] loss: 0.488
[Epoch : 10, Iter:  1562] loss: 0.4

[Epoch : 1, Iter:  1562] loss: 1.280
[Epoch : 2, Iter:  1562] loss: 0.905
[Epoch : 3, Iter:  1562] loss: 0.741
[Epoch : 4, Iter:  1562] loss: 0.612
[Epoch : 5, Iter:  1562] loss: 0.513
Accuracy on test data
Accuracy of plane : 79.00 %
Accuracy of   car : 80.70 %
Accuracy of  bird : 63.20 %
Accuracy of   cat : 45.50 %
Accuracy of  deer : 61.50 %
Accuracy of   dog : 80.50 %
Accuracy of  frog : 76.90 %
Accuracy of horse : 74.00 %
Accuracy of  ship : 80.90 %
Accuracy of truck : 76.40 %
71.85999999999999
Accuracy on training data
Accuracy of plane : 89.89 %
Accuracy of   car : 93.51 %
Accuracy of  bird : 85.02 %
Accuracy of   cat : 67.96 %
Accuracy of  deer : 81.49 %
Accuracy of   dog : 91.98 %
Accuracy of  frog : 88.33 %
Accuracy of horse : 88.12 %
Accuracy of  ship : 92.34 %
Accuracy of truck : 92.09 %
87.07179335021272
[Epoch : 6, Iter:  1562] loss: 0.416
[Epoch : 7, Iter:  1562] loss: 0.349
[Epoch : 8, Iter:  1562] loss: 0.291
[Epoch : 9, Iter:  1562] loss: 0.259
[Epoch : 10, Iter:  156

[Epoch : 1, Iter:  1562] loss: 1.163
[Epoch : 2, Iter:  1562] loss: 0.885
[Epoch : 3, Iter:  1562] loss: 0.783
[Epoch : 4, Iter:  1562] loss: 0.703
[Epoch : 5, Iter:  1562] loss: 0.634
Accuracy on test data
Accuracy of plane : 70.00 %
Accuracy of   car : 85.30 %
Accuracy of  bird : 65.10 %
Accuracy of   cat : 54.70 %
Accuracy of  deer : 75.00 %
Accuracy of   dog : 72.00 %
Accuracy of  frog : 83.60 %
Accuracy of horse : 69.30 %
Accuracy of  ship : 90.80 %
Accuracy of truck : 88.40 %
75.41999999999999
Accuracy on training data
Accuracy of plane : 79.54 %
Accuracy of   car : 91.86 %
Accuracy of  bird : 79.37 %
Accuracy of   cat : 69.66 %
Accuracy of  deer : 86.11 %
Accuracy of   dog : 81.05 %
Accuracy of  frog : 87.93 %
Accuracy of horse : 81.70 %
Accuracy of  ship : 96.26 %
Accuracy of truck : 92.76 %
84.62253320549279
[Epoch : 6, Iter:  1562] loss: 0.560
[Epoch : 7, Iter:  1562] loss: 0.490
[Epoch : 8, Iter:  1562] loss: 0.418
[Epoch : 9, Iter:  1562] loss: 0.369
[Epoch : 10, Iter:  156

Finally we can look at the trends of the accuracy on seen and unseen data during training. What considerations can you make looking at the plot? What can be the causes of these trends?


In [ ]:
#@title Accuracy across epochs plot
fig = go.Figure()
for lenet_name, lenet_score in net_scores.items():
    x = np.arange(len(lenet_score))
    fig.add_trace(go.Scatter(x=x, y=lenet_score, mode='lines', name=lenet_name + ' test'))
for lenet_name, lenet_score in net_tr_scores.items():
    x = np.arange(len(lenet_score))
    fig.add_trace(go.Scatter(x=x, y=lenet_score, mode='lines', name=lenet_name + ' train'))

fig.update_layout(
        title='Accuracy across epochs',
        xaxis_title="epoch / test frequency",
        yaxis_title="Accuracy")

fig.show()
print(f'test frequency: {test_freq}')

NameError: name 'net_scores' is not defined

**SPOILER** (make your own considerations before reading over)

Before any comparison between models, we notice a strong memorization; not only the accuracy on training data is much higher than the one on test data, but also reaching 100% is suspicious. In all likelihood the models have memorized the labels of some training samples. After all the size of the training set is very limited for this kind of problem.

Concerning the two regulirizers we are working on, both of them improved the results of the vanilla architecture, with batchnorm performing slightly better than dropout.

> **EXERCISE** After these results a bunch of questions arise. Can we mitigate the memorization effect augmenting the training dataset with some transformations? What happen if we rise the dropout coefficient? And if we apply dropout also on the input layer? What is the performance of a model with both batch normalization and dropout, in which order you should place them? Does the performance of `MonteCarloDropoutLeNet` increase if we rise the number of predictions from 20 to 50 or 100?



### Experimenting with uncertainty

In this section we will see first-hand the ensemble behind a single neural net trained with dropout, looking closer at the predictions of each model in the ensemble.

First of all we load a pretrained `MonteCarloDropoutLeNet`.

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-0HX-AriuL8JOmknPHPvWgjdFfDhmSCx' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-0HX-AriuL8JOmknPHPvWgjdFfDhmSCx" -O MonteCarloDropoutLeNet.pt && rm -rf /tmp/cookies.txt

--2024-09-12 00:50:02--  https://docs.google.com/uc?export=download&confirm=&id=1-0HX-AriuL8JOmknPHPvWgjdFfDhmSCx
Resolving docs.google.com (docs.google.com)... 142.250.4.100, 142.250.4.101, 142.250.4.102, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1-0HX-AriuL8JOmknPHPvWgjdFfDhmSCx&export=download [following]
--2024-09-12 00:50:03--  https://drive.usercontent.google.com/download?id=1-0HX-AriuL8JOmknPHPvWgjdFfDhmSCx&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.130.132, 2404:6800:4003:c01::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.130.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 55142763 (53M) [application/octet-stream]
Saving to: ‘MonteCarloDropoutLeNet.pt’

MonteCarloDropoutLe 100%[===================>]  52.59

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [ ]:
model = FullDropoutLeNet()
model.load_state_dict(torch.load('/content/MonteCarloDropoutLeNet.pt'))
model = model.to(device)

<ipython-input-12-e5e1bb278c24>:2: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



In [ ]:
model.train()
test(model, is_MCDO=True)

Accuracy on test data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Accuracy of plane : 81.30 %
Accuracy of   car : 88.60 %
Accuracy of  bird : 69.20 %
Accuracy of   cat : 55.70 %
Accuracy of  deer : 71.80 %
Accuracy of   dog : 70.10 %
Accuracy of  frog : 85.40 %
Accuracy of horse : 82.90 %
Accuracy of  ship : 87.40 %
Accuracy of truck : 85.20 %
77.76


77.76

Since we want to test the uncertainty of the predictions of our model, we want to craft a difficult classification task.  
Let's find two samples visually similar but with a different label in the test set.

In [ ]:
inputs = [testset[i][0] for i in range(36)]
class_idx = [testset[i][1] for i in range(36)]

visualize_samples(inputs)

What about the dog and the horse at the beginnning of the second line?

If we flip the horse horizontally they are even closer. Let's prepare a smooth transition between the two images in 12 steps, we are going to analyze the predictions of our model on each of these.

In [ ]:
inputs_or = [testset[12][0], torch.flip(testset[13][0], dims=[2])]  #flip on dimension 2 to flip horizontally
inputs = [inputs_or[0] * (1 - i) + inputs_or[1] * i for i in np.linspace(0,1,num=12)]
visualize_samples(inputs)

Let's start looking at the labels predicted on each of these images by `MonteCarloDropoutLeNet`, so averaging over 100 predictions.

Monte Carlo Dropout requires several predictions for the same sample to work, making test time more computationally expensive and time consuming. Nevertheless we have GPUs, and we can place copies of the same sample in a batch. Until the batch fits in the GPU memory we can make these multiple predictions without slowdowns.

> **EXERCISE** Parallelize the following code removing the second `for`. Can you parallelize it even more removing also the first `for`?

In [ ]:
model.train()
with torch.no_grad():
    for i, sample in enumerate(inputs):
        sample = sample.to(device)
        output = torch.zeros(10, device=device)

        for j in range(100):
            single_output = model(sample.unsqueeze(0))
            output += single_output[0] / 100.

        class_index = torch.argmax(output).item()
        print(i, classes[class_index])


0 dog
1 dog
2 dog
3 dog
4 dog
5 dog
6 horse
7 horse
8 horse
9 horse
10 horse
11 horse


In [ ]:
# @title Solution 👀
model.train()
with torch.no_grad():
    for i, sample in enumerate(inputs):
        sample = sample.to(device)
        sample = sample.repeat(100, 1, 1, 1)
        outputs = model(sample)
        output = torch.einsum('il -> l', F.softmax(outputs, dim=1) / 100.)
        class_index = torch.argmax(output).item()
        print(i, classes[class_index])

0 dog
1 dog
2 dog
3 dog
4 dog
5 dog
6 horse
7 horse
8 horse
9 horse
10 horse
11 horse


Finally we want to look at the distribution of activations on the last layer on the neurons corresponding to the dog and horse classes, both before and after performing the softmax.

In [ ]:
model.train()
all_outputs = []
all_soft_outputs = []
with torch.no_grad():
    for sample in inputs:
        sample = sample.to(device)
        sample = sample.repeat(100, 1, 1, 1)
        outputs = model(sample)
        soft_outputs = F.softmax(outputs, dim=1)
        all_outputs.append(outputs.to('cpu').numpy())
        all_soft_outputs.append(soft_outputs.to('cpu').numpy())

Let's make a nice plot

In [ ]:
visualize_samples(inputs)
for k, output_sequence in enumerate([all_outputs, all_soft_outputs]):
    if k == 0:
        title = 'outputs of the neurons corresponding to dog and horse of the last layer before softmax (100 forward passes with dropout)'
    else:
        title = 'outputs of the neurons corresponding to dog and horse of the last layer after softmax (100 forward passes with dropout)'
    fig = go.Figure()
    ndfp = output_sequence[0].shape[0]  # number_of_different_forward_passes
    x_dogs = np.zeros(len(output_sequence) * ndfp)
    y_dogs = np.zeros(len(output_sequence) * ndfp)
    x_horses = np.zeros(len(output_sequence) * ndfp)
    y_horses = np.zeros(len(output_sequence) * ndfp)
    for i, output in enumerate(output_sequence):
        x_dogs[i * ndfp: (i+1) * ndfp] += i
        y_dogs[i * ndfp: (i+1) * ndfp] = output[:,5]
        x_horses[i * ndfp: (i+1) * ndfp] += i
        y_horses[i * ndfp: (i+1) * ndfp] = output[:,7]

    fig.add_trace(go.Scatter(x=x_dogs, y=y_dogs,
                        mode='markers',
                        name='dogs',
                        marker=dict(
                            size=50,
                            opacity=0.1,
                            symbol='line-ew',
                            line=dict(width=6, color='deepskyblue'))))
    fig.add_trace(go.Scatter(x=x_horses, y=y_horses,
                        mode='markers',
                        name='horses',
                        marker=dict(
                            size=50,
                            opacity=0.1,
                            symbol='line-ew',
                            line=dict(width=6, color='salmon'))))
    fig.update_layout(
        title=title,
        xaxis_title="image",
        yaxis_title="neuron activation",
        xaxis_type='category')

    fig.show()


First of all let's look at the results on the central images *before* the softmax, the more ambiguous ones. As expected the activations of the dog and horse neurons are quite close; the distributions of these activations over the ensemble of models are overlapping. Differently from the first and last images where the two distributions are well separated.

Look now at the activations *after* the softmax, despite the higher ambiguity of the central images, we have several models in the ensemble with the maximum activation of 1. It should be now clear that we can not attribute a 100% confidence to a classification based on a softmax close to one.

The softmax output of a class can be arbitrarily high if its mean is far from the means of the other classes.

Instead we can evaluate the **uncertainty** of our predictions using Monte Carlo dropout and looking at the **overlapping of the distributions** of activations before the softmax.
